<div style="display: flex;  height: 300px;">
    <img src="resources/langchain.jpeg"  style="margin-left:auto; margin-right:auto"/>
</div>

# Effektiv programvareutvikling med hjelp av LLM-er

***LangChain*** er et rammeverk designet for å forenkle utviklingen av applikasjoner ved hjelp av store språkmodeller (LLM-er). Det kan brukes til ulike formål som dokumentanalyse og sammendrag, chatbots og kodeanalyse.

<div style="display: flex;  height: 600px;">
    <img src="resources/langchain_components.png"  style="margin-left:auto; margin-right:auto"/>
</div>

# **Viktig**: Kjør cellen nedenfor for å laste inn OpenAI API-nøkkelen for resten av notebooken

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
from typing import Optional
_ = load_dotenv(find_dotenv()) # read local .env file

# 1. Chains

Å bruke en LLM isolert er greit for enkle applikasjoner, men for mer komplekse applikasjoner kreves det å kjede sammen LLM-er. Enten med hverandre eller med andre komponenter.

# Hvorfor navnet LangChain?

<div style="display: flex;  height: 80px;">
    <img src="resources/simple_chain.png"  style="margin-left:auto; margin-right:auto"/>
</div>

Hver operasjon på input eller output til en LLM betraktes som et steg i en chain. Hovedformålet med LangChain er evnen til å "kjede" sammen flere steg og lage en kompleks applikasjon som krever flere operasjoner eller kall til forskjellige språkmodeller. 

LangChain gir et enkelt grensesnitt som gjør det mulig å bygge slike applikasjoner og logikk. En kan også bygge slike applikasjoner uten LangChain, men med store applikasjoner kan kodestørrelse og vedlikehold raskt komme ut av kontroll. Derfor er bruk av LangChain og de medfølgende verktøyene anbefalt for utvikling av LLM-applikasjoner.

### Lag en enkel chain

En enkel chain kan være et steg bestående av å kalle LLM'en med en prompt og be om en spesifikk output. Langchain tilbyr et enkelt og effektivt grensesnitt kalt **L**ang**C**hain **E**xpression **L**anguage (***LCEL***) som forenkler prosessen med å lage chains.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Fortell meg om {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4)
output_parser = StrOutputParser()

# Chain definition is the pipline of putting all the above elements together like below:
chain = prompt | model | output_parser

En kan deretter kalle chainen med en inputvariabel ved hjelp av <code>invoke</code>.

In [ ]:
print(chain.invoke({"topic":"Sverige"}))

**Det kan ta litt tid før output blir generert. Kanskje foretrekker du koden nedenfor:**

In [ ]:
for chunk in chain.stream({"topic":"Bergen"}):
    print(chunk, end="", flush=True)

## Langchain tilbyr nyttige verktøy 'out of the box'

Mange av de tidligere funksjonalitetene vi har sett er allerede tilgjengelige på en enkel måte med LangChain. Du kan dermed utvikle applikasjonene dine raskt og enkelt ved å bruke disse funksjonene. Her er et eksempel på teksttagging og uthenting av nøkkelinformasjon ved hjelp av OpenAI-funksjoner og LangChain.

1. Lag først en klasse som inneholder informasjonen du vil trekke ut eller tagger du vil tilordne til teksten

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

2. Deretter kan du opprette en tagging chain ved å bruke `.bind()`-metoden i modellen:

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Define the LLM with temprature 
model = ChatOpenAI(temperature=0)

# Create the prompt for the language model.
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and tag the sentence"),
    ("user", "{input}")
])

# Convert the Tagging class to OpenAI function format
tagging_functions = [convert_pydantic_to_openai_function(Tagging)] 

# Add the function to the model using the .bind() method
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [ ]:
# Check the description of the model with binded function.
model_with_functions.kwargs['functions']

3. Lag en chain med modellen ovenfor og den gitte ekstraksjonsprompten

In [ ]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

4. Test modellen med noen eksempelsetninger på forskjellige språk. (Kan være kreativ her)

In [ ]:
tagging_chain.invoke({"input": "I love langchain"})

In [ ]:
tagging_chain.invoke({"input": "til helvete med Langchain"})

In [ ]:
tagging_chain.invoke({"input": "qué es langchain?"})

Prøv med dine egne setninger:

In [ ]:
tagging_chain.invoke({"input": ...})

## 2. Agenter

Hovedideen med agenter er å bruke en språkmodell til å velge en sekvens av handlinger. I chains er en sekvens av handlinger hardkodet (i kode). I agenter brukes en språkmodell som en resonnementmotor til å bestemme hvilke handlinger som skal utføres og i hvilken rekkefølge.

<div style="display: flex;  height: 260px;">
    <img src="resources/llm_agent.png"  style="margin-left:auto; margin-right:auto"/>
    <img src="resources/langgraph.png"  style="margin-left:auto; margin-right:auto"/>

    
</div>

## Lag en agent ved hjelp av ***LangGraph***:

LangGraph er et bibliotek for å bygge stateful, multi-actor applikasjoner med LLM-er, bygget på toppen av (og ment å brukes med) LangChain. Det utvider ***LangChain Expression Language*** med evnen til å koordinere flere chains over flere trinn av beregning på en syklisk måte. Generell inspirasjon er hentet fra Pregel og Apache Beam, og nåværende grensesnitt er inspirert av ***NetworkX***.


**Eksempeloppgaven er å lage en agent som har tilgang til Wikipedia som en utvidet kunnskapskilde. Agenten vil søke på nettsiden hvis den trenger en ekstern kilde for å svare på input.**

1. Først definerer vi tools/verktøy som vi ønsker å bruke. I dette eksempelet vil vi bruke et søkeverktøy innebygd i Langchain for Wikipedia. Det er imidlertid veldig enkelt å lage dine egne tools/verktøy og de kan tilpasses forskjellige behov og scenario.

In [ ]:
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun, WikipediaAPIWrapper
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.prebuilt import ToolExecutor
## Define the input schema for the tool function. In this case the tool accepts a query string as input.

class Wikipeida_tool_arg_schema(BaseModel):
    """Input for the Wikipeida tool."""

    query: str = Field(description="search query to look up")

# A list of tools are provided for the agent to utilize
tools = [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(), args_schema=Wikipeida_tool_arg_schema)]

# Wrap these tools in a simple LangGraph ToolExecutor a class that calls that tool, and returns the output
tool_executor = ToolExecutor(tools)


Deretter må vi laste inn modellen vi vil bruke. Det er viktig at modellen oppfyller to kriterier:

1. Den bør fungere med lister av meldinger. Vi vil representere alle agenttilstander i form av meldinger, så den må fungere godt med dem.

2. Den bør fungere med OpenAI-funksjoner. Dette betyr at den enten bør være en OpenAI-modell eller en modell som eksponerer et lignende grensesnitt.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import format_tool_to_openai_function


# We will set streaming=True so that we can stream tokens
model = ChatOpenAI(temperature=0, streaming=True)

# Converting the LangChain tools into the format for OpenAI function calling, and then bind them to the model class.
functions = [format_tool_to_openai_function(t) for t in tools]
model_with_functions = model.bind_functions(functions)

## Lage grafer

<div style="display: flex;  height: 552px;">
    <img src="resources/langgraph_agent.png"  style="margin-left:auto; margin-right:auto"/>
    
</div>

### Agenttilstand-definisjon

Hovedgrafen i `langgraph` er `StatefulGraph`. Denne grafen er parametrisert av et tilstands-objekt som sendes rundt til hver node. Hver node returnerer deretter operasjoner for å oppdatere tilstanden. Disse operasjonene kan enten SETTE spesifikke attributter for tilstanden (f.eks. overskrive eksisterende verdier) eller LEGGE TIL blant de eksisterende attributtene. Om det skal 'settes' eller 'legges til' angis ved å annotere tilstands-objektet du konstruerer grafen med.

For dette eksempelet vil vi spore en liste over meldinger. Vi vil at hver node bare skal legge til meldinger i listen. Derfor vil vi bruke en `TypedDict` med én nøkkel (`messages`) og annotere den slik at det alltid legges til `messages`-attributtet.

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

# Define the state properties that will be passed through the graph
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Definer nodefunksjonene til grafen

Vi må nå definere forskjellige noder i grafen. I `LangGraph` kan en node enten være en function eller en runnable. Det er to hovednoder vi trenger for dette:

1. ***Agenten***: ansvarlig for å bestemme hvilke (om noen) handlinger som skal utføres.
2. ***En funksjon***: for å kalle tools/verktøy. Hvis agenten bestemmer seg for å utføre en handling vil denne noden deretter utføre handlingen.


La oss definere nodene, samt en funksjon som er definert inne i nodene.

In [ ]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model_with_functions.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    print(action)
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

## Koble sammen nodene og lag graf-strukturen

Vi må også definere noen kanter. Noen kanter kan være betingede ettersom utgangen av en node kan være en av flere alternative stier. Stien som tas er ikke kjent før noden kjøres (LLM'en bestemmer).

1. ***Betinget kant***: Etter et agentkall er det to alternativer.   
    **a.** Hvis agenten bestemmer en aksjon bør funksjonen for å kalle tools/verktøy kalles.   
    **b.** Hvis agenten sier at den er ferdig, så bør den avslutte.   

    

2. ***Normal kant***: Etter at et tool/verktøy er kalt, bør den alltid gå tilbake til agenten for å bestemme neste steg/handling.


 
La oss koble sammen nodene definert ovenfor med riktige kanter:

In [ ]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# Define the function that determines the route in the conditional edge
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


# We now add a conditional edge
workflow.add_conditional_edges(
    # We set 'agent' as the starting Node.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings output from the above function, and the values are other nodes' names.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

Graf-strukturen er nå komplett og vi kan kalle den ved hjelp av inputmeldinger:

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="Hello how are you?")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="What is langchain?")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

# TODO: Agent med flere tools/verktøy 

## Prøv å legge til flere tools/verktøy for agenten

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool

class CalculatorInput(BaseModel):
    #TODO Define the schema of the input to the tool
    a: ... = Field(description="...") # TODO: fill in the blanks with proper object types and description of parameters for the first number
    b: ... = Field(description="...") # second number
    op: ... = Field(description="...") # operation type

@tool(args_schema=CalculatorInput)
def calculator_tool(a: ..., b: ..., op: ...) -> int:
    """...""" #fill in the description of the tool. This description helps the agent to decide when to use the tool based on the input prompt.
    return ...



## TODO: Legg til nye tools/verktøy definert over til i listen av tools/verktøy for LangGraph-agenten.

The aim is for the agent to have access to both tools of wikipedia search and the custom tool defined below.

In [ ]:
# A list of tools are provided for the agent to utilize
tools = [...]

# Wrap these tools in a simple LangGraph ToolExecutor a class that calls that tool, and returns the output
tool_executor = ToolExecutor(tools)

# Bind the language model to the tools defined above.
# Use the code above in the agent definition section to get inspiration for this part. The aim is for the tools list to have 2 values :
# 1. Wikipeida tool and 2. the custom calculator tool 3. Optional custom tool
model = ChatOpenAI(temperature=0, streaming=True)
functions = [format_tool_to_openai_function(t) for t in tools]
model_with_functions = model.bind_functions(functions)

In [ ]:
tools

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="What is 2375 times 452?")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")